In [ ]:
import requests
import oyez_api
import numpy as np
import pprint
import pickle

from pymongo import MongoClient
client = MongoClient()

In [ ]:
#Cases HREF: https://api.oyez.org/cases?filter=term:2017&per_page=100
#Saving 1 Oral HREF for reference: #https://api.oyez.org/case_media/oral_argument_audio/24307

#cases_total is dict of lists of all URLS for cases between 2000 and 2018. 

cases_total={}
for year in range(2000,2018):
    cases_year = []
    cases_year = oyez_api.cases(year)
    cases_total[year]=cases_year

cases_total[np.random.randint(2000,2018)][6] #random case to check on data

Where Am I inserting this data? scotus collection within scotus_db in MongoDB.  

Unstructured database will allow flexible data addition

In [ ]:
scotus_db = client.scotus_db
scotus = scotus_db.scotus_collection

for year in range(2000,2018):

    case_year = cases_total[year]
    print(f"Year: {year}")
    for case in case_year:

        #grab voting results
        majority_list, minority_list,other_list = oyez_api.voting_decisions(case)

        if oyez_api.get_oral_href(case) is None:
            print(f"Case: {case['name']} has no oral arguments")
        else:
            full_oral_page = oyez_api.get_oral_href(case)
            for section in full_oral_page:
                for turns in section['turns']:
                    for text_block in turns['text_blocks']:
#                         print(text_block)

                        #Set "role"
                        if turns['speaker'] is None:
                            role = "other_speaker"
                            speaker = "other_speaker"
                            identifier = "other_speaker"
                        elif turns['speaker']['roles'] is None:
                            role = "other_speaker"
                            speaker = ""
                            identifier = ""
                        elif not turns['speaker']['roles']:

                            role = "none"
                            speaker = turns['speaker']['name']
                            identifier = turns['speaker']['identifier']
                        else:

                            role = turns['speaker']['roles'][0]['type']
                            speaker = turns['speaker']['name']
                            identifier = turns['speaker']['identifier']

                        #Set "voting_result"
                        voting_result = oyez_api.justice_voting(majority_list,minority_list,other_list,speaker)

                        #Create entry dict
                        #Enter dictionary into scotus MongoDB
                        entry={}
                        entry={
                            'case':case['ID']
                            ,'docket_num':case['docket_number']
                            ,'term':case['term']
                            ,'case_name':case['name']
                            ,'speaker':speaker
                            ,'role': role
                            ,'identifier': identifier
                            ,'voting_result': voting_result
                            ,'text':text_block['text']
                            ,'start':text_block['start']
                            ,'stop':text_block['stop']
                                }
                        scotus.insert_one(entry) 